In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(subscription_id="04bf6e4a-98b1-41c0-b2b3-c7362661b532", name="azure-ml-udacity-workspace", resource_group="azure-ml-udacity")
exp = Experiment(workspace=ws, name="hyperdrive-experiment")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

compute_cluster_name = "azureml-compute1"
try:
    compute_cluster = ComputeTarget(workspace=ws, name=compute_cluster_name)
    print('Found existing cluster')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V3', max_nodes=1)
    compute_cluster = ComputeTarget.create(ws, compute_cluster_name, compute_config)

compute_cluster.wait_for_completion(show_output=True)

Found existing cluster

Running


In [3]:
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive import choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "C": uniform(0.5, 2.0),
    "max_iter": choice(100, 200, 500)
})

# Specify a Policy
termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(
    source_directory="./", 
    compute_target=compute_cluster,
    entry_script="./train.py",
    pip_requirements_file_path="./requirements.txt")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                           hyperparameter_sampling=ps,
                           policy=termination_policy,
                           primary_metric_name='Accuracy',
                           primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                           max_total_runs=10,
                           max_concurrent_runs=1)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [32]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.widgets import RunDetails

hyperdrive_run = exp.submit(config=hyperdrive_config)

RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [36]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print('\nAccuracy:', best_run_metrics['Accuracy'])
print('\nModel', best_run_metrics['Model'])

joblib.dump(best_run_metrics['Model'], "hyperdrive_model.pkl")

Best Run Id:  HD_df21aa59-5573-4921-b4b6-38033b962b61_2

Accuracy: 0.9156297420333839

Model LogisticRegression(C=1.817070726530091, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)


['hyperdrive_model.pkl']

In [15]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data_url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(data_url)

In [16]:
from train import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [17]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=20,
    task="classification",
    primary_metric="accuracy",
    training_data=x.join(y),
    label_column_name="y",
    n_cross_validations=3)

In [18]:
# Submit your automl run

automl_exp = Experiment(ws, "automl-experiment")
automl_run = automl_exp.submit(config=automl_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running on local machine
Parent Run ID: AutoML_06fc3f9b-a3a4-4ac3-9855-68e8c9d52620

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/Auto

In [21]:
# Retrieve and save your best automl model.

# automl_best_run = automl_run.get_best_child()
automl_best_run, automl_model = automl_run.get_output()
automl_best_run_metrics = automl_best_run.get_metrics()

print('Best Run Id: ', automl_best_run.id)
print('\n Accuracy:', automl_best_run_metrics['accuracy'])

print(automl_model)

joblib.dump(automl_model, "automl_model.pkl")

Best Run Id:  AutoML_06fc3f9b-a3a4-4ac3-9855-68e8c9d52620_26

 Accuracy: 0.9161456336645278
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                l1_ratio=None,
                                                                                                max_iter=100,
                                                                               

['automl_model.pkl']

In [43]:
amlcompute = AmlCompute(ws, compute_cluster_name)
amlcompute.delete()